* MAE	
Помогает оценить абсолютную ошибку: насколько в среднем число в предсказании разошлось с реальным числом.	Удобно интерпретировать. Измеряется в тех же единицах, что и целевой признак. Несильно искажается при наличии выбросов.	Не поможет, если необходимо сравнить модели, предсказывающие одно и то же по разным признакам.	mean_absolute_error()
* MAPE	
Помогает абстрагироваться от конкретных чисел и оценить абсолютную ошибку в процентах.	Легко интерпретировать. Используется в задачах, где неизвестно, какое значение метрики считать приемлемым.	Плохо подходит для задач, в которых важны конкретные единицы измерений. Лучше использовать в паре с MAE, чтобы знать абсолютную ошибку и её значение в процентах.	mean_absolute_percentage_error()
* MSE	
Интерпретации нет. Используется в задачах, где критически важны большие ошибки, например при предсказании координат полёта.	Каждая ошибка вносит свой квадратичный штраф, большие расхождения между предсказанием и истиной увеличивают штраф.	Измеряется в квадратах единиц, поэтому менее доступна для понимания. Искажается при наличии выбросов. Не поможет, если нужно сравнить модели, предсказывающие одно и то же по разным признакам.	mean_squared_error()
* RMSE	
Можно трактовать как стандартное отклонение предсказаний от истинных ответов. Используется в тех же задачах, что и MSE.	Имеет те же преимущества, что и MSE, но более удобна для понимания (измеряется в тех же единицах, что и целевая переменная).	Не поможет, если нужно сравнить модели, предсказывающие одно и то же по разным признакам.	Отдельная функция отсутствует, но можно извлечь корень из результата функции mean_squared_error().
* r2
Помогает понять, какую долю разнообразия (дисперсии) смогла уловить модель в данных. Позволяет сравнить, насколько модель лучше, чем простое предсказание средним.	Можно сравнивать модели, обученные на разных признаках. Легко оценить качество модели: измеряется от  до 1. Удовлетворительным показателем считается показатель выше 0.5.	Чувствительна к добавлению новых данных. Чувствительна к выбросам, так как основана на MSE.	r2_score()

# РАСЧЁТ МЕТРИК НА PYTHON

Настало время проверить качество построенных нами ранее моделей линейной регрессии: lr_lstat и lr_full.

Весь набор функций для вычисления метрик в sklearn находится в модуле metrics. Давайте его импортируем:

В каждую из функций достаточно передать правильные ответы и предсказания, и функция вернёт рассчитанную метрику.

Примечание. Для расчёта метрики RMSE нет специальной функции, однако мы знаем, что для её расчёта достаточно извлечь квадратный корень из MSE.

Из-за особенностей реализации функция mean_absolute_percentage_error() возвращает результат не в процентах, а в долях. Чтобы отобразить результат в процентах, необходимо умножить его на 100.

Давайте вычислим метрики и выведем их на экран, округлив до третьего знака после запятой. Начнём с модели lr_lstat: сделаем предсказание на основании признака LSTAT и передадим истинные и предсказанные медианные цены в функции для расчёта метрик:

In [1]:
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации
from sklearn import linear_model #линейные модели
from sklearn import metrics #метрики
%matplotlib inline
plt.style.use('seaborn-v0_8')

In [2]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
boston_data = pd.read_csv('data/housing.csv', header=None, delimiter=r"\s+", names=column_names)


In [4]:
X = boston_data[['LSTAT']] #матрица наблюдений
y = boston_data['MEDV'] #вектор правильных ответов


In [5]:
#Создаём объект класса LinearRegression
lr_lstat = linear_model.LinearRegression()
#Обучаем модель — ищем параметры по МНК
lr_lstat.fit(X, y)
 

LinearRegression()

In [6]:
#Составляем список факторов (исключили целевой столбец)
features = boston_data.drop('MEDV', axis=1).columns
#Составляем матрицу наблюдений X и вектор ответов y
X = boston_data[features]
y = boston_data['MEDV']
#Создаём объект класса LinearRegression
lr_full = linear_model.LinearRegression()
#Обучаем модель — ищем параметры по МНК
lr_full.fit(X, y)

LinearRegression()

In [7]:
#Делаем предсказание по признаку LSTAT
y_predict_lstat = lr_lstat.predict(boston_data[['LSTAT']])
#Рассчитываем MAE
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y, y_predict_lstat)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y, y_predict_lstat))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict_lstat) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_predict_lstat)))
 

MAE score: 4.505 thou. $
RMSE score: 6.203 thou. $
MAPE score: 21.352 %
R2 score: 0.544


Проделываем ту же самую операцию для второй модели линейной регрессии, lr_full:



In [8]:
#Делаем предсказание по всем признакам
y_predict_full = lr_full.predict(boston_data[features])
#Рассчитываем MAE
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y, y_predict_full)))
#Рассчитываем RMSE
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y, y_predict_full))))
#Рассчитываем MAPE
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict_full) * 100))
#Рассчитываем коэффициент детерминации
print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_predict_full)))
 

MAE score: 3.271 thou. $
RMSE score: 4.679 thou. $
MAPE score: 16.417 %
R2 score: 0.741


In [9]:
y_true = [1.23, 2.35, 2.75]
y_pred = [1.01, 12.3, 2.74]

np.sqrt(metrics.mean_squared_error(
    y_true,
    y_pred
    )
)

5.746042116100439

In [10]:
y_true = [22.4, 20.6, 23.9, 22.0, 11.9]
y_pred = [20.5, 20.2, 20.3, 19.0, 11.0]

metrics.r2_score(y_true, y_pred)

0.707747874730212

НЕДОСТАТКИ АНАЛИТИЧЕСКОГО РЕШЕНИЯ

Ранее мы с вами рассмотрели, что такое модель линейной регрессии, и научились рассчитывать её параметры с помощью аналитического подхода — метода наименьших квадратов.

Метод наименьших квадратов позволяет очень просто получить коэффициенты , подставив таблицу в формулу. Вот, собственно, и всё «обучение». 

→ Существует теорема Гаусса-Маркова, которая говорит о том, что, если выполнены все условия теоремы, МНК всегда находит оптимальные оценки параметров. Мы ещё вернемся к этой теореме, когда будем говорить об МНК в модулях по линейной алгебре.

Казалось бы, относительно простая математика: надо всего лишь перемножить матрицы между собой и получить ответ. Особенно простой эта задача должна быть для компьютера. Но нам так кажется, поскольку мы ранее не сталкивались с матричным умножением и будем говорить о нём только в модулях по линейной алгебре.

Оказывается, у такого простого подхода есть один большой минус — это работа с большим количеством признаков.

Давайте внимательно посмотрим на операцию обращения матриц (возведение в степень -1):

Таблица  имеет размер , то есть у неё  строк и  столбцов. Таблица  — это результат транспонирования матриц (замены строк и столбцов местами), то есть её размерность — . Забегая вперёд, отметим, что по правилам умножения матриц результат умножения будет иметь размерность , где  — это число столбцов.

А теперь представим, что у нас не 13 признаков, а 1300. То есть матрица  имеет размерность . Но мы делаем вычисления на компьютере, так ведь? Сложностей быть не должно, но... В модулях по линейной алгебре мы увидим, что обращение матриц — очень ресурсозатратная операция. У неё кубическая сложность, то есть если размер матрицы — , то на её обращение понадобится  операций. Для нашего примера это !

Обращение матриц больших размеров может стать очень трудоёмким процессом при работе с большими объёмами данных.

Второй недостаток МНК — это невозможность инкрементального обучения, или обучения в режиме реального времени.

Что это такое?

Представьте, что мы построили модель, но собираемся её уточнять в процессе эксплуатации. К нам приходят всё новые данные, и мы должны изменять параметры модели, подстраиваясь под новые зависимости.

Если мы используем метод fit() для модели LinearRegression и передадим в него новые данные, то коэффициенты модели будут рассчитаны по новым данным, а прошлые наблюдения будут забыты. То есть придётся добавлять данные в таблицу и переобучать модель на всех доступных данных ещё раз.

Первая и вторая проблемы решаются с помощью замены аналитического МНК на численные методы, такие как градиентный спуск.

Третий недостаток МНК больше теоретический и заключается в том, что матрица  в результате вычислений может не существовать. Это связано с математическими особенностями вычисления обратной матрицы, которые мы рассмотрим далее в курсе. 

Причина этой проблемы — мультиколлинеарность факторов (сильная корреляционная связь). Из-за этого коэффициенты линейной регрессии становятся слишком большими и модель становится неустойчивой. 

Проблема решается с помощью регуляризации.